# Lecture_4_Setting_Parallel_Computing_Configuration_for_SUMMA_in_Rivanna

* Before starting this notebook, you need to check your Jupyter Kernel as `pyrhessys`.

## 1. Import pyrhessys

In [1]:
import pyrhessys as pr
import os

## 2. Change the relative directory to set RHESSys configuration in Rivanna environment

#### * When you already changed RHESSys configuration file of relative path in Rivanna, you don't need to repeat it again

### 2.1 Change the relative directory to set RHESSys configuration in worldfile 

In [2]:
worldfiles = os.path.join(os.getcwd(), 'worldfiles', 'worldfile.hdr')
worldfiles

'/sfs/qumulo/qhome/yc5ef/pyrhessys/worldfiles/worldfile.hdr'

In [3]:
read_worldfiles = open(worldfiles,'r').read()
print(read_worldfiles[0:200])

1 num_basin_files
/sfs/qumulo/qhome/yc5ef/pyrhessys/defs/basin_basin.def basin_default_filename
1 num_hillslope_files
/sfs/qumulo/qhome/yc5ef/pyrhessys/defs/hill_setting1.def hillslope_default_filenam


In [ ]:
# check your original path of configuration file in CyberGIS-Jupyter for water
current_string = '/home/jovyan/work/diff_run/model'

In [ ]:
os.getcwd()

In [ ]:
# set current configuration path in Rivanna
new_string = '/sfs/qumulo/qhome/yc5ef/pyrhessys'

In [ ]:
pr.utils.replace_string(worldfiles, current_string, new_string)

### 2.2 Change the relative directory to indicate rhessys configuration in clim file 

In [ ]:
clim = os.path.join(os.getcwd(), 'clim', 'cwt.base')
clim

In [28]:
read_clim = open(clim,'r').read()
print(read_clim[50:500])

18129.311971 y_coordinate 
638.0 z_coordinate 
2.0 effective_lai 
22.9 	screen_height 
/sfs/qumulo/qhome/yc5ef/pyrhessys/clim/na annual_climate_prefix 
0 
/sfs/qumulo/qhome/yc5ef/pyrhessys/clim/na monthly_climate_prefix 
0 
/sfs/qumulo/qhome/yc5ef/pyrhessys/clim/cwt daily_climate_prefix 
0 
/sfs/qumulo/qhome/yc5ef/pyrhessys/clim/na hourly_climate_prefix 
0 


In [8]:
replace_string(clim, current_string, new_string)

## 3. Start Parallel Computing

### 3.1 Create "parallel input" folder to store parallel execution command text file

In [14]:
import os
parallel = os.path.join(os.getcwd(), 'parallel_input')
if not os.path.exists(parallel):
    os.mkdir(parallel)
else:
    pass

### 3.2 Set Combination of Ensemble Simulations

In [5]:
import numpy as np
def safe_arange(start, stop, step):
    a = np.arange(start, stop, step)
    result =[]
    for i in a:
        par = round(i, 10)
        result = np.append(result, par)
    return result

In [11]:
# set each paremters to create ensemble simulation
param_options = {
    's1': safe_arange(1, 20, 0.5),  
    's2': safe_arange(0.4, 2.4, 0.1)
}

In [4]:
# set each paremters to create ensemble simulation
param_options = {
    'gw1': safe_arange(0.01, 0.09, 0.05),
    'gw2': safe_arange(0.01, 0.09, 0.05),
    's1': safe_arange(1, 20, 10),  
    's2': safe_arange(0.4, 2.4, 1.5),
    's3': safe_arange(0.4, 2.4, 1.5),
    'snowEs': safe_arange(0.3, 1.0, 0.4),
    'snowTs': safe_arange(0.5, 2.0, 0.8),
    'sv1': safe_arange(1.0, 3.0, 0.8),
    'sv2': safe_arange(100, 200, 60.0),
    'svalt1': safe_arange(0.5, 1.5, 0.6),
    'svalt2': safe_arange(0.5, 1.5, 0.6)
}

In [12]:
def parameter_product(list_config):
    return {'++'+'++'.join('{}={}'.format(k, v) for k, v in d.items())+'++':
            {'parameters': d} for d in pr.utils.product_dict(**list_config)}

In [14]:
# create config for parameters ensemble
config = parameter_product(param_options)
# check the number of ensemble combinations
len(config)

760

### 3.3 Create run.slurm to execute RHESSys with parallel

In [15]:
current_dir = os.getcwd()

In [20]:
%%writefile {current_dir}/run.slurm
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH -t 06:00:00
#SBATCH -p standard
#SBATCH --array=0-800

inputdir=/sfs/qumulo/qhome/yc5ef/pyrhessys/parallel_input

# get all files
shopt -s nullglob
allfiles=($inputdir/*.txt)

# each task handles a distint group of the text files
fileindex=$SLURM_ARRAY_TASK_ID
while [[ $fileindex -lt ${#allfiles[@]} ]]; do
    # pass the text file to be processed as first argument
    file=${allfiles[fileindex]}
    eval bash "$file"
    fileindex=$((fileindex + $SLURM_ARRAY_TASK_COUNT))
done

Overwriting /sfs/qumulo/qhome/yc5ef/pyrhessys/run.slurm


### 3.4 Set pyrhessys Simulation Object and parameters

In [21]:
executable = os.path.join(os.getcwd(), 'RHESSysEastCoast', 'rhessysEC.7.2')
r = pr.Simulation(executable, os.getcwd())

In [22]:
# read parameter values from parameter_meta.json 
r.parameters

{'version': 'rhessysEC.7.2',
 'start_date': '2008 01 01 01',
 'end_date': '2012 12 31 01',
 'gw1': '0.06',
 'gw2': '0.06',
 's1': '1.0',
 's2': '1.9',
 's3': '1.9',
 'snowEs': '0.7',
 'snowTs': '1.3',
 'sv1': '2.6',
 'sv2': '100.0',
 'svalt1': '0.5',
 'svalt2': '1.1',
 'locationid': '0',
 'basin_id': 1,
 'hillslope_id': 1,
 'zone_id': 1,
 'patch_id': 1}

In [23]:
# set parameter values 
r.parameters['version'] = 'rhessysEC.7.2'
r.parameters['start_date'] = '2008 01 01 01'
r.parameters['end_date'] = '2012 12 31 01'
r.parameters['gw1'] = '0.06'
r.parameters['gw2'] = '0.06'
r.parameters['s1'] = '1.0'
r.parameters['s2'] = '1.9'
r.parameters['s3'] = '1.9'
r.parameters['snowEs'] = '0.7'
r.parameters['snowTs'] = '1.3'
r.parameters['sv1'] = '2.6'
r.parameters['sv2'] = '100.0'
r.parameters['svalt1'] = '0.5'
r.parameters['svalt2'] = '1.1'
r.parameters['locationid'] = '0'

In [24]:
# read parameter values from parameter_meta.json 
r.parameters

{'version': 'rhessysEC.7.2',
 'start_date': '2008 01 01 01',
 'end_date': '2012 12 31 01',
 'gw1': '0.06',
 'gw2': '0.06',
 's1': '1.0',
 's2': '1.9',
 's3': '1.9',
 'snowEs': '0.7',
 'snowTs': '1.3',
 'sv1': '2.6',
 'sv2': '100.0',
 'svalt1': '0.5',
 'svalt2': '1.1',
 'locationid': '0',
 'basin_id': 1,
 'hillslope_id': 1,
 'zone_id': 1,
 'patch_id': 1}

### 3.5 Create Parallel execution command file in paralle_input folder

In [25]:
r.parallel_job(executable, start_date, end_date, config, os.getcwd()+'/parallel_input')

cd /sfs/qumulo/qhome/yc5ef/pyrhessys/RHESSysEastCoast; ./rhessysEC.7.2 -st 2008 01 01 01 -ed 2012 12 31 01 -b -gwtoriparian -t /sfs/qumulo/qhome/yc5ef/pyrhessys/tecfiles/tec_daily.txt -w /sfs/qumulo/qhome/yc5ef/pyrhessys/worldfiles/worldfile -whdr /sfs/qumulo/qhome/yc5ef/pyrhessys/worldfiles/worldfile.hdr -r /sfs/qumulo/qhome/yc5ef/pyrhessys/flows/subflow.txt /sfs/qumulo/qhome/yc5ef/pyrhessys/flows/surfflow.txt -pre /sfs/qumulo/qhome/yc5ef/pyrhessys/output/++s1=1.0++s2=0.4++  -gw 0.06 0.06  -s 1.0 0.4 1.9  -snowEs 0.7  -snowTs 1.3  -sv 2.6 100.0 -svalt 0.06 0.06 
cd /sfs/qumulo/qhome/yc5ef/pyrhessys/RHESSysEastCoast; ./rhessysEC.7.2 -st 2008 01 01 01 -ed 2012 12 31 01 -b -gwtoriparian -t /sfs/qumulo/qhome/yc5ef/pyrhessys/tecfiles/tec_daily.txt -w /sfs/qumulo/qhome/yc5ef/pyrhessys/worldfiles/worldfile -whdr /sfs/qumulo/qhome/yc5ef/pyrhessys/worldfiles/worldfile.hdr -r /sfs/qumulo/qhome/yc5ef/pyrhessys/flows/subflow.txt /sfs/qumulo/qhome/yc5ef/pyrhessys/flows/surfflow.txt -pre /sfs/qumul

['++s1=1.0++s2=0.4++',
 '++s1=1.0++s2=0.5++',
 '++s1=1.0++s2=0.6++',
 '++s1=1.0++s2=0.7++',
 '++s1=1.0++s2=0.8++',
 '++s1=1.0++s2=0.9++',
 '++s1=1.0++s2=1.0++',
 '++s1=1.0++s2=1.1++',
 '++s1=1.0++s2=1.2++',
 '++s1=1.0++s2=1.3++',
 '++s1=1.0++s2=1.4++',
 '++s1=1.0++s2=1.5++',
 '++s1=1.0++s2=1.6++',
 '++s1=1.0++s2=1.7++',
 '++s1=1.0++s2=1.8++',
 '++s1=1.0++s2=1.9++',
 '++s1=1.0++s2=2.0++',
 '++s1=1.0++s2=2.1++',
 '++s1=1.0++s2=2.2++',
 '++s1=1.0++s2=2.3++',
 '++s1=1.5++s2=0.4++',
 '++s1=1.5++s2=0.5++',
 '++s1=1.5++s2=0.6++',
 '++s1=1.5++s2=0.7++',
 '++s1=1.5++s2=0.8++',
 '++s1=1.5++s2=0.9++',
 '++s1=1.5++s2=1.0++',
 '++s1=1.5++s2=1.1++',
 '++s1=1.5++s2=1.2++',
 '++s1=1.5++s2=1.3++',
 '++s1=1.5++s2=1.4++',
 '++s1=1.5++s2=1.5++',
 '++s1=1.5++s2=1.6++',
 '++s1=1.5++s2=1.7++',
 '++s1=1.5++s2=1.8++',
 '++s1=1.5++s2=1.9++',
 '++s1=1.5++s2=2.0++',
 '++s1=1.5++s2=2.1++',
 '++s1=1.5++s2=2.2++',
 '++s1=1.5++s2=2.3++',
 '++s1=2.0++s2=0.4++',
 '++s1=2.0++s2=0.5++',
 '++s1=2.0++s2=0.6++',
 '++s1=2.0+